In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Indetify Bridges with Abnormal Years

In [2]:
xls = pd.ExcelFile('ASNA 2020 Case Competition -- Database.xlsx')
# https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook
df_map = {}
i=0
for sheet_name in xls.sheet_names:
    if (i==0):
        i=1
        continue
    df_map[sheet_name] = xls.parse(sheet_name)
    df_map[sheet_name]['year']=int(sheet_name)
data_all=pd.concat(df_map, ignore_index=True)

In [3]:
data_all['year_code']=data_all['Bridge ID'].str.slice(start=3, stop=10)

In [4]:
year_key=data_all.groupby(['year_code'])['Year Built'].agg(pd.Series.mode).to_frame().T.to_dict('list')

In [5]:
def correct_year(b):
    return year_key[b['year_code']][0]

In [6]:
data_all['correct_year']=data_all.apply(correct_year, axis=1)

In [7]:
incorrect=data_all[data_all['correct_year']!=data_all['Year Built']]

In [8]:
incorrect['Bridge ID'].value_counts()

HLN999X86202700E01153    2458
FBA631L18702126N04706     233
Name: Bridge ID, dtype: int64

## Final Model

In [9]:
clean_data=pd.read_csv('newtb2.csv')

In [10]:
clean_data=clean_data.replace('N', 0)
zr=np.zeros((1,len(clean_data)))[0]
zr[clean_data['Structural improvement Needed']!=0]=1
clean_data['need_cons']=zr

In [11]:
clean_data.head()

,Bridge ID,Road Type,Year Built,Number of Lanes,Avg Daily Traffic,Operational Status,Structure Material,Deck Material,Bridge Design,Number of Spans,Bridge Length,Road Width,Deck Width,Assessment,Structural improvement Needed,Lifetime,need_cons
0,FBA631L18702126N04706,5,1982,1,50,1,1,3,85,1,11.2,7.9,8.5,2,0,31,0.0
1,AVS013E67702643T01389,5,2011,2,2000,1,2,1,75,1,41.1,12.8,13.8,2,0,7,0.0
2,AVS013E67706985P09389,1,2011,3,200,1,1,1,75,2,80.2,16.2,17.2,2,0,7,0.0
3,AVS013E67709899C08982,5,2011,2,20,1,1,0,79,1,8.2,5.5,11.0,2,0,7,0.0
4,AVS014K98701615G09088,2,1901,2,255,1,2,3,75,1,8.2,6.7,7.0,3,0,110,0.0


In [12]:
count_no_sub = len(clean_data[clean_data['need_cons']==0])
pct_of_no_sub = count_no_sub/len(clean_data)
print("% of no Structural Improvement is", pct_of_no_sub*100)
pct_of_sub = 1-pct_of_no_sub
print("% of Structural Improvement", pct_of_sub*100)

% of no Structural Improvement is 38.87520831778723
% of Structural Improvement 61.124791682212766


In [13]:
y = clean_data['need_cons']
X = clean_data[['Road Type','Number of Lanes','Avg Daily Traffic','Operational Status', 'Bridge Length','Assessment','Lifetime']]
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [14]:
logreg = LogisticRegression(random_state=0, solver='liblinear')
logreg.fit(X_train, y_train)
print(logreg.score(X_train, y_train))
print(logreg.score(X_valid, y_valid))

0.6658596444680287
0.676350611879415


## Making Prediction

In [15]:
dic={'Road Type':[],'Number of Lanes':[],'Avg Daily Traffic':[],'Operational Status':[], 'Bridge Length':[],'Assessment':[],'Lifetime':[]}

In [16]:
for t in range(1,6):
    for nl in range(2,18,4):
        for avg in range(100000,800000,200000):
            for op in range(1,6):
                for leng in range(1000,6000,1000):
                    for ass in [1,3,5,7]:
                        for lt in range(10,120,15):
                            dic['Road Type'].append(t)
                            dic['Number of Lanes'].append(nl)
                            dic['Avg Daily Traffic'].append(avg)
                            dic['Operational Status'].append(op)
                            dic['Bridge Length'].append(leng)
                            dic['Assessment'].append(ass)
                            dic['Lifetime'].append(lt)

In [17]:
df_out=pd.DataFrame.from_dict(dic)
df_out['Prob of re-construction']=logreg.predict_proba(df_out)[:,1]
df_out.to_excel('Prob_model.xlsx',index=False)

## Total Combination

In [18]:
dic={'Road Type':[],'Number of Lanes':[],'Avg Daily Traffic':[],'Operational Status':[], 'Bridge Length':[],'Assessment':[],'Lifetime':[]}

In [19]:
for t in range(1,6):
    for nl in range(2,18,4):
        for avg in range(100000,800000,200000):
            for op in range(1,6):
                for leng in range(1000,6000,1000):
                    for ass in [1,3,5,7]:
                        for lt in range(0,119):
                            dic['Road Type'].append(t)
                            dic['Number of Lanes'].append(nl)
                            dic['Avg Daily Traffic'].append(avg)
                            dic['Operational Status'].append(op)
                            dic['Bridge Length'].append(leng)
                            dic['Assessment'].append(ass)
                            dic['Lifetime'].append(lt)

In [20]:
df_out=pd.DataFrame.from_dict(dic)

In [21]:
df_out['Prob of structural repair/replacement']=logreg.predict_proba(df_out)[:,1]

In [22]:
# df_out.to_excel('Prob_model_total.xlsx',index=False)